<a href="https://colab.research.google.com/github/frrabelo/image2braile/blob/main/image2Braile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-cloud-vision google-colab


In [ ]:
import io
from google.cloud import vision
from google.colab.patches import cv2_imshow
import cv2
from google.colab import files
from google.drive import Drive


In [ ]:
#Carregue a API Key do Google Gemini de um arquivo seguro no Google Drive
from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/MyDrive/path/to/your/api_key.json', 'r') as f:
    api_key = json.load(f)


In [ ]:
#Crie uma função para carregar a imagem da câmera ou do armazenamento local:
def load_image(image_type="camera"):
    """
    Carrega uma imagem da câmera ou do armazenamento local.

    Args:
        image_type (str): Tipo de imagem ("camera" ou "local").

    Returns:
        numpy.ndarray: Imagem carregada.
    """
    if image_type == "camera":
        # Captura imagem da câmera
        cap = cv2.VideoCapture(0)
        ret, frame = cap.read()
        cap.release()
        if not ret:
            raise Exception("Falha ao capturar imagem da câmera")
        return frame
    elif image_type == "local":
        # Carrega imagem do armazenamento local
        uploaded = files.upload()
        for filename, content in uploaded.items():
            image = cv2.imdecode(np.frombuffer(content, np.uint8), cv2.IMREAD_COLOR)
            return image
    else:
        raise ValueError("Tipo de imagem inválido: {}".format(image_type))


In [ ]:
#Crie uma função para realizar o reconhecimento de texto na imagem usando a API Key do Google Gemini:
def recognize_text(image, api_key):
    """
    Realiza o reconhecimento de texto em uma imagem usando a API Key do Google Gemini.

    Args:
        image (numpy.ndarray): Imagem para reconhecimento de texto.
        api_key (dict): API Key do Google Gemini.

    Returns:
        list: Lista de dicionários com informações do texto reconhecido.
    """
    client = vision.ImageAnnotatorClient()

    # Converter imagem em formato compatível com a API
    content = image.tobytes()
    image = vision.types.Image(content=content)

    # Criar solicitação de detecção de texto
    features = [vision.types.Feature(type=vision.types.Feature.TEXT_DETECTION)]
    request = vision.types.ImageAnnotationRequest(features=features, image=image)

    # Realizar a detecção de texto
    response = client.image_annotate(requests=[request])
    text_blocks = response.text_annotations

    # Extrair informações do texto
    recognized_text = []
    for text_block in text_blocks:
        for paragraph in text_block.paragraphs:
            for word in paragraph.words:
                recognized_text.append({
                    "bounds": word.bounding_box,
                    "text": word.text
                })
    return recognized_text


In [ ]:
#Função para Converter Texto com Fonte do Google Drive:
#Crie uma função para converter o texto reconhecido em um formato específico usando uma fonte do Google Drive:

def convert_text_to_


In [ ]:
#Função para Converter Texto com Fonte do Google Drive:

def convert_text_to_document(recognized_text, font_path):
    """
    Converte o texto reconhecido em um documento usando uma fonte do Google Drive.

    Args:
        recognized_text (list): Lista de dicionários com informações do texto reconhecido.
        font_path (str): Caminho para a fonte no Google Drive.

    Returns:
        str: Texto convertido em formato de documento.
    """
    # Carregar a fonte do Google Drive
    with open(font_path, 'rb') as f:
        font_data = f.read()

    # Criar objeto de fonte
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_size = 12

    # Criar imagem vazia para renderizar o texto
    image = np.zeros((500, 500, 3), dtype=np.uint8)

    # Desenhar o texto na imagem
    for text_info in recognized_text:
        x, y, w, h = text_info['bounds']
        text = text_info['text']
        cv2.putText(image, text, (x, y + h), font, font_size, (0, 0, 0), thickness=2)

    # Converter imagem em texto
    _, buffer = cv2.imencode('.png', image)
    io_obj = io.BytesIO(buffer)

    # Carregar biblioteca de processamento de texto
    import docx

    # Criar novo documento do Word
    document = docx.Document()

    # Adicionar parágrafo ao documento
    paragraph = document.add_paragraph()

    # Adicionar imagem com texto ao parágrafo
    paragraph.add_run().add_picture(io_obj, width=document.page_width, height=document.page_height)

    # Salvar o documento como texto
    document_text = document.as_text()

    return document_text


In [ ]:
#Função Principal:

def main():
    # Carregar imagem
    image = load_image(image_type="camera")  # "camera" ou "local"

    # Reconhecer texto na imagem
    recognized_text = recognize_text(image, api_key)

    # Converter texto em documento usando fonte do Google Drive
    font_path = '/content/drive/MyDrive/path/to/your/font.ttf'  # Caminho para a fonte
    document_text = convert_text_to_document(recognized_text, font_path)

    # Exibir o texto convertido
    print(document_text)

if __name__ == "__main__":
    main()


**Observações:**

*   List item Substitua api_key pelo conteúdo do seu arquivo JSON da API Key do Google Gemini.
*   List item Substitua /content/drive/MyDrive/path/to/your/font.ttf pelo caminho real da fonte no seu Google Drive.
*   List item Você pode modificar a função convert_text_to_document para formatar o texto da maneira desejada (tamanho da fonte, espaçamento, etc.).
*   List item Este código é apenas um exemplo e pode ser adaptado para atender às suas necessidades específicas.






O código que forneci fornece uma estrutura básica para criar um projeto no Google Colab que utiliza a API Key do Google Gemini para realizar o reconhecimento de texto em uma imagem e converter o texto extraído em um formato específico usando uma fonte do Google Drive.

**Algumas considerações adicionais:**

*   **Precisão do Reconhecimento de Texto:** A precisão do reconhecimento de texto dependerá da qualidade da imagem e da complexidade do texto. Imagens com baixa iluminação, ruído ou texto em fontes complexas podem ter menor precisão de reconhecimento.
*   **Limites da API Key do Google Gemini:** A API Key do Google Gemini possui limites de uso. Certifique-se de verificar os limites e custos da API antes de usá-la em um projeto de produção.
*   **Processamento de Texto Avançado:** O código fornecido realiza apenas a conversão básica do texto em um formato de documento. Você pode usar bibliotecas de processamento de texto mais avançadas para realizar tarefas como formatação, correção ortográfica e análise de sentimento.
*   **Interface do Usuário:** Você pode aprimorar o projeto criando uma interface do usuário que permita aos usuários selecionar a imagem, escolher a fonte e visualizar o texto convertido de forma mais intuitiva.

Recomendo que você explore os recursos adicionais mencionados na resposta anterior para se aprofundar em cada tópico e adaptar o código às suas necessidades específicas.